# Prep

Setting up some prior functionality

In [ ]:
!git clone https://github.com/thepycoder/detr.git

In [3]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

1.10.0 True


# Initialise ClearML

In [4]:
# Preferably run this in a terminal if you can, but if in a (colab) notebook, the input is not recognised properly. Press enter once, to give clearml-init an empty input and then fill in the fields one at a time.
!pip install clearml
!clearml-init

ClearML SDK setup process
Configuration file already exists: /root/clearml.conf
Leaving setup, feel free to edit the configuration file.


In [ ]:
# Download the data from the clearML server (I just had to run `clearml-data get` to get a local copy of my data)
!wget https://files.community.clear.ml/dragon_detector/dragon_coco.731383ca315447a8bbaf91c4a8bdecf2/artifacts/data/dataset.731383ca315447a8bbaf91c4a8bdecf2.zip
!unzip dataset.731383ca315447a8bbaf91c4a8bdecf2.zip

In [5]:
# This is for myself
!clearml-data get --id 731383ca315447a8bbaf91c4a8bdecf2 --copy dragon_data

clearml-data - Dataset Management & Versioning CLI
Download dataset id 731383ca315447a8bbaf91c4a8bdecf2
2021-11-14 19:26:19,226 - clearml.storage - INFO - Downloading: 5.00MB / 57.74MB @ 2.91MBs from https://files.community.clear.ml/dragon_detector/dragon_coco.731383ca315447a8bbaf91c4a8bdecf2/artifacts/data/dataset.731383ca315447a8bbaf91c4a8bdecf2.zip
2021-11-14 19:26:20,812 - clearml.storage - INFO - Downloading: 10.00MB / 57.74MB @ 3.15MBs from https://files.community.clear.ml/dragon_detector/dragon_coco.731383ca315447a8bbaf91c4a8bdecf2/artifacts/data/dataset.731383ca315447a8bbaf91c4a8bdecf2.zip
2021-11-14 19:26:22,515 - clearml.storage - INFO - Downloading: 15.00MB / 57.74MB @ 2.94MBs from https://files.community.clear.ml/dragon_detector/dragon_coco.731383ca315447a8bbaf91c4a8bdecf2/artifacts/data/dataset.731383ca315447a8bbaf91c4a8bdecf2.zip
2021-11-14 19:26:24,222 - clearml.storage - INFO - Downloading: 20.00MB / 57.74MB @ 2.93MBs from https://files.community.clear.ml/dragon_detecto

The rest of the clearml-specific code is added in the `main.py` file and will keep track of every time it is ran from here. Keeping track of all the variables, arguments, metrics, model files and so on.

# Load a model

Since the data in our dragon dataset is limited, our model should be pretrained. 

This means getting a pretrained model and chopping off the class head, so we can retrain that part only.

In [6]:
%cd detr
# Get pretrained weights
checkpoint = torch.hub.load_state_dict_from_url(
            url='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth',
            map_location='cpu',
            check_hash=True)

# Remove class weights
del checkpoint["model"]["class_embed.weight"]
del checkpoint["model"]["class_embed.bias"]

# SaveOGH
torch.save(checkpoint, 'detr-r50_no-class-head.pth')

[Errno 2] No such file or directory: 'detr'
/project/final_product/detr


# Dataset

Our dataset should be loadable as a COCO format

This allows us to use the pycocotools to load the data dict for the main python script

In [7]:
dataset_file = "dragons" # alternatively, implement your own coco-type dataset loader in datasets and add this "key" to datasets/__init__.py

dataDir='/project/final_product/dragon_data' # should lead to a directory with a train and val folder as well as an annotations folder
num_classes = 2 # this int should be the amount of classes + 1

outDir = 'outputs'
resume = "detr-r50_no-class-head.pth"

# Training

We use the main.py script to run our training

In [8]:
!python main.py \
  --dataset_file $dataset_file \
  --coco_path $dataDir \
  --output_dir $outDir \
  --resume $resume \
  --num_classes $num_classes \
  --lr 1e-5 \
  --lr_backbone 1e-6 \
  --epochs 100

ClearML Task: created new task id=9342ccae460c415cbd5b9182d3f51fa3
ClearML results page: https://app.community.clear.ml/projects/8131d2f23983461f8354156be95bc4f1/experiments/9342ccae460c415cbd5b9182d3f51fa3/output/log
Not using distributed mode
git:
  sha: b0915f1ee533687935b3ee4cc5059a472c25b90a, status: has uncommited changes, branch: master

Namespace(aux_loss=True, backbone='resnet50', batch_size=2, bbox_loss_coef=5, clip_max_norm=0.1, coco_panoptic_path=None, coco_path='/project/final_product/dragon_data', dataset_file='dragons', dec_layers=6, device='cuda', dice_loss_coef=1, dilation=False, dim_feedforward=2048, dist_url='env://', distributed=False, dropout=0.1, enc_layers=6, eos_coef=0.1, epochs=100, eval=False, frozen_weights=None, giou_loss_coef=2, hidden_dim=256, lr=1e-05, lr_backbone=1e-06, lr_drop=200, mask_loss_coef=1, masks=False, nheads=8, num_classes=1, num_queries=100, num_workers=2, output_dir='outputs', position_embedding='sine', pre_norm=False, remove_difficult=Fals

# Results

Quick and easy overview of the training results

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path(outDir)]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest)   

# Visualise results

In [ ]:
model = torch.hub.load('facebookresearch/detr',
                       'detr_resnet50',
                       pretrained=False,
                       num_classes=num_classes)

checkpoint = torch.load('outputs/checkpoint.pth',
                        map_location='cpu')

model.load_state_dict(checkpoint['model'],
                      strict=False)

model.eval()

In [ ]:
import matplotlib.pyplot as plt

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

finetuned_classes = ['dragon']

def plot_finetuned_results(pil_img, prob=None, boxes=None):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    if prob is not None and boxes is not None:
      for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{finetuned_classes[cl]}: {p[cl]:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

In [ ]:
import torchvision.transforms as T

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b


def filter_bboxes_from_outputs(outputs,
                               threshold=0.1):
  
  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  return probas_to_keep, bboxes_scaled

def run_worflow(my_image, my_model):
  # mean-std normalize the input image (batch-size: 1)
  img = transform(my_image).unsqueeze(0)

  # propagate through the model
  outputs = my_model(img)

  for threshold in [0.7, 0.2]:
    
    probas_to_keep, bboxes_scaled = filter_bboxes_from_outputs(outputs,
                                                              threshold=threshold)

    plot_finetuned_results(my_image,
                           probas_to_keep, 
                           bboxes_scaled)

In [ ]:
from PIL import Image

img_name = '/content/dragon_data/val/dsfgfdsfgdsfg.jpg'
im = Image.open(img_name)

run_worflow(im,
            model)

In [ ]:
import os
for img_name in os.listdir("/content/dragon_data/val"):
    im = Image.open(img_name)
    run_worflow(im, model)